# Addition Game: Dynamic Programming Solution

## Game Definition (from Blackwell's "Game Theory and Statistical Decisions")

> *The parameters of the game k and N are given. Player I and Player II alternately choose integers, each choice being one of the integers 1,...,k and each choice made with the knowledge of all preceding choices. As soon as the sum of the chosen integers exceeds N, the last player to choose loses the game and pays his opponent one unit.*

In this notebook, we solve the Addition game using **Dynamic Programming** (DP). Unlike sampling-based methods (Monte Carlo, Q-learning), DP computes optimal values and policies exactly by solving the **Bellman equations** directly. This is possible because the game has:
- A finite state space (sums 0 to N+k)
- Known transition dynamics (deterministic)
- Complete model of the game


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, List, Dict, Optional
from tqdm import tqdm

# Set style for visualizations
plt.style.use('dark_background')
plt.rcParams['font.family'] = 'monospace'
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.facecolor'] = '#1e1e2e'
plt.rcParams['figure.facecolor'] = '#11111b'
plt.rcParams['axes.edgecolor'] = '#6c7086'
plt.rcParams['axes.labelcolor'] = '#89b4fa'
plt.rcParams['xtick.color'] = '#a6adc8'
plt.rcParams['ytick.color'] = '#a6adc8'
plt.rcParams['text.color'] = '#cdd6f4'
plt.rcParams['grid.color'] = '#313244'
plt.rcParams['grid.alpha'] = 0.5


## 1. Bellman Equations for Two-Player Zero-Sum Games

For a two-player zero-sum game, we define the **minimax value** V(s, player) as the expected payoff to Player I when both players play optimally from state s.

### State Representation
- State s = (current_sum, current_player)
- current_sum ∈ {0, 1, ..., N+k}
- current_player ∈ {0 (Player I), 1 (Player II)}

### Terminal States
When sum > N, the player who just moved loses:
- V(s > N, just_moved=I) = -1 (Player I just lost)
- V(s > N, just_moved=II) = +1 (Player II just lost, so Player I wins)

### Bellman Optimality Equations

**For Player I (maximizer):**
$$V(s, \text{Player I}) = \max_{a \in \{1,...,k\}} V(s+a, \text{Player II})$$

**For Player II (minimizer):**
$$V(s, \text{Player II}) = \min_{a \in \{1,...,k\}} V(s+a, \text{Player I})$$

These equations express the minimax principle: Player I maximizes, Player II minimizes.


## 2. Game Environment


In [ ]:
@dataclass
class GameState:
    """Represents the current state of the Addition game."""
    current_sum: int
    current_player: int  # 0 for Player I, 1 for Player II
    
    def __hash__(self):
        return hash((self.current_sum, self.current_player))
    
    def __eq__(self, other):
        return (self.current_sum == other.current_sum and 
                self.current_player == other.current_player)


class AdditionGame:
    """The Addition game environment."""
    
    def __init__(self, k: int, N: int):
        self.k = k
        self.N = N
        self.reset()
    
    def reset(self) -> GameState:
        """Reset the game to initial state."""
        self.state = GameState(current_sum=0, current_player=0)
        self.history: List[int] = []
        self.done = False
        self.winner = None
        return self.state
    
    def get_valid_actions(self) -> List[int]:
        """Returns list of valid actions (1 to k)."""
        return list(range(1, self.k + 1))
    
    def step(self, action: int) -> Tuple[GameState, float, float, bool]:
        """Execute an action and return (new_state, reward_p1, reward_p2, done)."""
        if self.done:
            raise ValueError("Game is already over!")
        
        self.history.append(action)
        current_player = self.state.current_player
        new_sum = self.state.current_sum + action
        
        if new_sum > self.N:
            self.done = True
            self.winner = 1 - current_player
            reward_p1 = 1.0 if self.winner == 0 else -1.0
            reward_p2 = -reward_p1
        else:
            reward_p1 = 0.0
            reward_p2 = 0.0
        
        self.state = GameState(
            current_sum=new_sum,
            current_player=1 - current_player
        )
        
        return self.state, reward_p1, reward_p2, self.done


## 3. Value Iteration Algorithm

**Value Iteration** solves the Bellman equations iteratively until convergence:

1. Initialize V(s) = 0 for all states
2. Repeat until convergence:
   - For each state s where it's Player I's turn: V(s) ← max_a V(s+a)
   - For each state s where it's Player II's turn: V(s) ← min_a V(s+a)
3. Extract optimal policy from converged values

For the Addition game, we can solve this exactly with **backward induction** since the game has a natural ordering (higher sums come "later" in the game).


In [ ]:
class DynamicProgrammingSolver:
    """Solves the Addition game using Dynamic Programming (Value Iteration).
    
    Computes optimal values V(s) and optimal policies for both players
    by solving the Bellman equations.
    """
    
    def __init__(self, k: int, N: int):
        self.k = k
        self.N = N
        
        # Value function: V[sum][player] = value to Player I
        # We need values for sums 0 to N+k (terminal states)
        self.V = np.zeros((N + k + 1, 2))
        
        # Q-values: Q[sum][player][action] = value
        self.Q = np.zeros((N + k + 1, 2, k))
        
        # Optimal policy: policy[sum][player] = optimal action
        self.policy = np.zeros((N + 1, 2), dtype=int)
        
        # Initialize terminal values
        self._init_terminal_values()
        
    def _init_terminal_values(self):
        """Initialize values for terminal states (sum > N)."""
        # When sum > N, the player who just moved loses
        # V is always from Player I's perspective
        for s in range(self.N + 1, self.N + self.k + 1):
            # If it's Player I's turn at s > N, that means Player II just moved
            # and caused sum to exceed N, so Player II loses, Player I wins
            self.V[s, 0] = +1  # Player I's turn at terminal -> Player I wins
            
            # If it's Player II's turn at s > N, Player I just caused overflow
            # so Player I loses
            self.V[s, 1] = -1  # Player II's turn at terminal -> Player I loses
    
    def solve(self, verbose: bool = True) -> dict:
        """Solve the game using backward induction (a form of Value Iteration).
        
        Since sum always increases, we can solve exactly by working backwards
        from terminal states.
        """
        iterations = 0
        
        # Process states from highest sum to lowest (backward induction)
        for s in range(self.N, -1, -1):
            iterations += 1
            
            for player in [0, 1]:  # 0 = Player I, 1 = Player II
                best_value = None
                best_action = None
                
                for a_idx, a in enumerate(range(1, self.k + 1)):
                    next_sum = s + a
                    next_player = 1 - player
                    
                    # Get value of next state
                    if next_sum > self.N:
                        # Terminal: current player loses
                        if player == 0:  # Player I moves and loses
                            value = -1
                        else:  # Player II moves and loses, Player I wins
                            value = +1
                    else:
                        value = self.V[next_sum, next_player]
                    
                    # Store Q-value
                    self.Q[s, player, a_idx] = value
                    
                    # Update best action based on player's objective
                    if best_value is None:
                        best_value = value
                        best_action = a
                    elif player == 0:  # Player I maximizes
                        if value > best_value:
                            best_value = value
                            best_action = a
                    else:  # Player II minimizes
                        if value < best_value:
                            best_value = value
                            best_action = a
                
                self.V[s, player] = best_value
                self.policy[s, player] = best_action
        
        if verbose:
            print(f"Dynamic Programming solved in {iterations} iterations")
            print(f"Game value (V(0, Player I)): {self.V[0, 0]:.2f}")
            if self.V[0, 0] > 0:
                print("Player I wins with optimal play!")
            elif self.V[0, 0] < 0:
                print("Player II wins with optimal play!")
            else:
                print("Game is a draw with optimal play!")
        
        return {
            'V': self.V.copy(),
            'Q': self.Q.copy(),
            'policy': self.policy.copy(),
            'game_value': self.V[0, 0]
        }
    
    def get_optimal_action(self, state: GameState) -> int:
        """Get optimal action for given state."""
        if state.current_sum > self.N:
            return 1  # Game is over, return dummy
        return self.policy[state.current_sum, state.current_player]
    
    def get_value(self, state: GameState) -> float:
        """Get value of state from Player I's perspective."""
        return self.V[min(state.current_sum, self.N + self.k), state.current_player]


## 4. Solving the Game

Let's solve the Addition game with parameters `k=3` and `N=10`.


## 5. Visualizing the Value Function and Optimal Policy


In [ ]:
def visualize_dp_solution(solver: DynamicProgrammingSolver, k: int, N: int):
    """Visualize the DP solution: values, Q-values, and optimal policy."""
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    fig.suptitle('Dynamic Programming Solution for Addition Game', 
                 fontsize=16, fontweight='bold', color='#89b4fa')
    
    sums = list(range(0, N + 1))
    actions = list(range(1, k + 1))
    
    # --- Row 1: Player I's perspective ---
    
    # Value function for Player I
    ax1 = axes[0, 0]
    values_p1 = [solver.V[s, 0] for s in sums]
    colors = ['#a6e3a1' if v > 0 else '#f38ba8' if v < 0 else '#f9e2af' for v in values_p1]
    ax1.bar(sums, values_p1, color=colors, edgecolor='white', linewidth=0.5)
    ax1.axhline(y=0, color='#6c7086', linewidth=1)
    ax1.set_xlabel('Current Sum')
    ax1.set_ylabel('Value')
    ax1.set_title('V(s, Player I) - Value Function', color='#89b4fa')
    ax1.set_ylim(-1.2, 1.2)
    ax1.grid(True, alpha=0.3)
    
    # Q-values for Player I
    ax2 = axes[0, 1]
    q_matrix_p1 = solver.Q[:N+1, 0, :]
    im2 = ax2.imshow(q_matrix_p1, aspect='auto', cmap='RdYlGn', vmin=-1, vmax=1)
    ax2.set_yticks(range(len(sums)))
    ax2.set_yticklabels(sums)
    ax2.set_xticks(range(k))
    ax2.set_xticklabels(actions)
    ax2.set_ylabel('Current Sum')
    ax2.set_xlabel('Action')
    ax2.set_title('Q(s, a, Player I)', color='#89b4fa')
    plt.colorbar(im2, ax=ax2, label='Q-value')
    
    for i in range(len(sums)):
        for j in range(k):
            color = 'white' if abs(q_matrix_p1[i, j]) > 0.5 else 'black'
            ax2.text(j, i, f'{q_matrix_p1[i, j]:.0f}', ha='center', va='center', 
                     color=color, fontsize=9, fontweight='bold')
    
    # Optimal policy for Player I
    ax3 = axes[0, 2]
    policy_p1 = solver.policy[:, 0]
    colors_policy = ['#f38ba8', '#f9e2af', '#a6e3a1']
    ax3.barh(sums, policy_p1, color=[colors_policy[a-1] for a in policy_p1], 
             edgecolor='white', linewidth=0.5)
    ax3.set_xlabel('Optimal Action')
    ax3.set_ylabel('Current Sum')
    ax3.set_title('Optimal Policy π*(s, Player I)', color='#89b4fa')
    ax3.set_xticks([1, 2, 3])
    ax3.set_xlim(0, k + 0.5)
    ax3.invert_yaxis()
    
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=colors_policy[i], label=f'Action {i+1}') for i in range(k)]
    ax3.legend(handles=legend_elements, loc='lower right', facecolor='#1e1e2e', edgecolor='#6c7086')
    
    # --- Row 2: Player II's perspective ---
    
    # Value function for Player II's turn (still from P1's perspective)
    ax4 = axes[1, 0]
    values_p2 = [solver.V[s, 1] for s in sums]
    colors = ['#a6e3a1' if v > 0 else '#f38ba8' if v < 0 else '#f9e2af' for v in values_p2]
    ax4.bar(sums, values_p2, color=colors, edgecolor='white', linewidth=0.5)
    ax4.axhline(y=0, color='#6c7086', linewidth=1)
    ax4.set_xlabel('Current Sum')
    ax4.set_ylabel('Value')
    ax4.set_title('V(s, Player II) - Value when P2 moves', color='#89b4fa')
    ax4.set_ylim(-1.2, 1.2)
    ax4.grid(True, alpha=0.3)
    
    # Q-values for Player II
    ax5 = axes[1, 1]
    q_matrix_p2 = solver.Q[:N+1, 1, :]
    im5 = ax5.imshow(q_matrix_p2, aspect='auto', cmap='RdYlGn', vmin=-1, vmax=1)
    ax5.set_yticks(range(len(sums)))
    ax5.set_yticklabels(sums)
    ax5.set_xticks(range(k))
    ax5.set_xticklabels(actions)
    ax5.set_ylabel('Current Sum')
    ax5.set_xlabel('Action')
    ax5.set_title('Q(s, a, Player II)', color='#89b4fa')
    plt.colorbar(im5, ax=ax5, label='Q-value')
    
    for i in range(len(sums)):
        for j in range(k):
            color = 'white' if abs(q_matrix_p2[i, j]) > 0.5 else 'black'
            ax5.text(j, i, f'{q_matrix_p2[i, j]:.0f}', ha='center', va='center', 
                     color=color, fontsize=9, fontweight='bold')
    
    # Optimal policy for Player II
    ax6 = axes[1, 2]
    policy_p2 = solver.policy[:, 1]
    ax6.barh(sums, policy_p2, color=[colors_policy[a-1] for a in policy_p2], 
             edgecolor='white', linewidth=0.5)
    ax6.set_xlabel('Optimal Action')
    ax6.set_ylabel('Current Sum')
    ax6.set_title('Optimal Policy π*(s, Player II)', color='#89b4fa')
    ax6.set_xticks([1, 2, 3])
    ax6.set_xlim(0, k + 0.5)
    ax6.invert_yaxis()
    ax6.legend(handles=legend_elements, loc='lower right', facecolor='#1e1e2e', edgecolor='#6c7086')
    
    plt.tight_layout()
    plt.show()

visualize_dp_solution(solver, K, N)


## 6. Explicit Bellman Equations

Let's write out the Bellman equations for each state explicitly.


In [ ]:
def print_bellman_equations(solver: DynamicProgrammingSolver, k: int, N: int):
    """Print the Bellman equations for each state."""
    
    print("BELLMAN EQUATIONS FOR THE ADDITION GAME")
    print("="*70)
    print(f"Parameters: k={k}, N={N}")
    print()
    
    print("TERMINAL STATES (sum > N):")
    print("-"*70)
    print("  V(s > N, Player I's turn) = +1  (Player II just lost)")
    print("  V(s > N, Player II's turn) = -1 (Player I just lost)")
    print()
    
    print("PLAYER I's BELLMAN EQUATIONS (maximizing):")
    print("-"*70)
    for s in range(N, -1, -1):
        q_terms = []
        for a in range(1, k + 1):
            next_s = s + a
            if next_s > N:
                q_terms.append(f"Q({s},{a})=-1")
            else:
                q_terms.append(f"Q({s},{a})=V({next_s},P2)={solver.V[next_s,1]:+.0f}")
        
        opt_a = solver.policy[s, 0]
        v_val = solver.V[s, 0]
        print(f"  V({s:2d}, P1) = max{{ {', '.join(q_terms)} }} = {v_val:+.0f}  [π*={opt_a}]")
    
    print()
    print("PLAYER II's BELLMAN EQUATIONS (minimizing):")
    print("-"*70)
    for s in range(N, -1, -1):
        q_terms = []
        for a in range(1, k + 1):
            next_s = s + a
            if next_s > N:
                q_terms.append(f"Q({s},{a})=+1")
            else:
                q_terms.append(f"Q({s},{a})=V({next_s},P1)={solver.V[next_s,0]:+.0f}")
        
        opt_a = solver.policy[s, 1]
        v_val = solver.V[s, 1]
        print(f"  V({s:2d}, P2) = min{{ {', '.join(q_terms)} }} = {v_val:+.0f}  [π*={opt_a}]")

print_bellman_equations(solver, K, N)


## 7. Dynamic Programming Agent

Let's create agents that use the DP-computed optimal policies.


In [ ]:
class DPAgent:
    """An agent that plays using the DP-computed optimal policy."""
    
    def __init__(self, player_id: int, solver: DynamicProgrammingSolver):
        self.player_id = player_id
        self.solver = solver
    
    def select_action(self, state: GameState, valid_actions: List[int], training: bool = False) -> int:
        """Select the optimal action computed by DP."""
        return self.solver.get_optimal_action(state)
    
    def get_value(self, state: GameState) -> float:
        """Get the value of a state."""
        return self.solver.get_value(state)


# Create DP agents
dp_agent1 = DPAgent(player_id=0, solver=solver)
dp_agent2 = DPAgent(player_id=1, solver=solver)

print("DP agents created using optimal policies from Value Iteration")


## 8. Watching Optimal Play

Let's watch the DP agents play a game, showing the value function at each step.


In [ ]:
def play_game_with_values(game: AdditionGame, agent1: DPAgent, agent2: DPAgent):
    """Play a game showing values at each step."""
    state = game.reset()
    agents = [agent1, agent2]
    names = ["Player I", "Player II"]
    
    print("="*70)
    print(f"OPTIMAL PLAY: Addition Game with k={game.k}, N={game.N}")
    print("="*70)
    print(f"Game Value V(0, P1) = {agent1.get_value(state):+.0f}")
    print("This means Player I " + ("WINS" if agent1.get_value(state) > 0 else "LOSES") + " with optimal play!")
    print("="*70)
    print()
    
    move = 1
    while not game.done:
        player = state.current_player
        agent = agents[player]
        name = names[player]
        
        # Get all Q-values for this state
        q_values = []
        for a in range(1, game.k + 1):
            next_sum = state.current_sum + a
            if next_sum > game.N:
                q_val = -1 if player == 0 else +1
            else:
                q_val = solver.V[next_sum, 1 - player]
            q_values.append(f"Q(a={a})={q_val:+.0f}")
        
        action = agent.select_action(state, game.get_valid_actions())
        value = agent.get_value(state)
        
        print(f"Move {move}: {name}'s turn at sum={state.current_sum}")
        print(f"         V(s={state.current_sum}, {name}) = {value:+.0f}")
        print(f"         Q-values: {', '.join(q_values)}")
        print(f"         {'Maximizing' if player == 0 else 'Minimizing'} -> chooses action {action}")
        print(f"         Sum: {state.current_sum} -> {state.current_sum + action}")
        
        state, _, _, done = game.step(action)
        
        if done:
            print(f"\n         *** SUM ({state.current_sum}) EXCEEDS {game.N}! ***")
            winner = "Player I" if game.winner == 0 else "Player II"
            print(f"\n  WINNER: {winner}")
        else:
            print()
        
        move += 1
    
    print()
    print("="*70)
    print(f"Game history: {game.history}")
    print("="*70)

play_game_with_values(game, dp_agent1, dp_agent2)


## 9. Analyzing Different Game Parameters

Let's use DP to solve various game configurations and see the pattern.


In [ ]:
def analyze_games_with_dp():
    """Solve multiple game configurations using DP."""
    
    print("DYNAMIC PROGRAMMING ANALYSIS OF DIFFERENT GAME CONFIGURATIONS")
    print("="*75)
    print(f"{'k':>3} {'N':>4} {'States':>8} {'V(0,P1)':>10} {'Winner':>12} {'P1 Policy[0]':>13}")
    print("-"*75)
    
    results = []
    
    configs = [
        (2, 5), (2, 6), (2, 7), (2, 8), (2, 9),
        (3, 8), (3, 9), (3, 10), (3, 11), (3, 12),
        (4, 12), (4, 13), (4, 14), (4, 15), (4, 16),
        (5, 18), (5, 19), (5, 20), (5, 21),
    ]
    
    for k, n in configs:
        solver = DynamicProgrammingSolver(k=k, N=n)
        solver.solve(verbose=False)
        
        game_value = solver.V[0, 0]
        winner = "Player I" if game_value > 0 else "Player II" if game_value < 0 else "Draw"
        states = 2 * (n + 1)
        first_action = solver.policy[0, 0]
        
        # Check if it matches modular arithmetic theory
        # Losing positions: s ≡ N (mod k+1)
        # P1 wins iff 0 is NOT a losing position, i.e., N mod (k+1) != 0
        theory_p1_wins = (n % (k + 1)) != 0
        theory_matches = (game_value > 0) == theory_p1_wins
        
        mark = "✓" if theory_matches else "✗"
        
        print(f"{k:>3} {n:>4} {states:>8} {game_value:>+10.0f} {winner:>12} {first_action:>13} {mark}")
        
        results.append({
            'k': k, 'N': n, 'value': game_value, 'winner': winner,
            'first_action': first_action, 'theory_matches': theory_matches
        })
    
    print("-"*75)
    print("Theory: Player I wins iff N mod (k+1) ≠ 0")
    print("✓ = DP result matches theoretical prediction")
    
    return results

results = analyze_games_with_dp()


## 10. Comparison: DP vs Learning Methods

Dynamic Programming computes exact optimal solutions, but requires a complete model of the environment. Let's compare the characteristics.


In [ ]:
def compare_methods():
    """Compare DP with learning-based methods."""
    
    comparison = """
    ╔═══════════════════════════════════════════════════════════════════════════════╗
    ║              COMPARISON: DYNAMIC PROGRAMMING vs LEARNING METHODS              ║
    ╠═══════════════════════════════════════════════════════════════════════════════╣
    ║ Aspect              │ Dynamic Programming │ Q-Learning/MC/GB                  ║
    ╠═══════════════════════════════════════════════════════════════════════════════╣
    ║ Model Required      │ Yes (complete)      │ No (model-free)                   ║
    ║ Computation         │ Polynomial in |S|   │ Depends on # episodes             ║
    ║ Solution Quality    │ Exact optimal       │ Approximate (converges)           ║
    ║ Sample Efficiency   │ Perfect (no samples)│ Requires many episodes            ║
    ║ Exploration         │ Not needed          │ Critical (ε-greedy, etc.)         ║
    ║ Online Learning     │ No                  │ Yes                               ║
    ║ Large State Spaces  │ Intractable         │ Can use function approximation    ║
    ║ Stochastic Games    │ Needs probabilities │ Learns from samples               ║
    ╚═══════════════════════════════════════════════════════════════════════════════╝
    """
    print(comparison)
    
    print("\nFor the Addition Game specifically:")
    print("-" * 60)
    print(f"State space size: 2 × (N+1) = {2*(N+1)} states")
    print(f"DP solves in: O(N × k) = O({N} × {K}) = {N*K} operations")
    print("Learning methods: Need ~50,000 episodes to converge")
    print()
    print("DP is clearly superior for this small, fully-known game.")
    print("But learning methods generalize to unknown/large environments!")

compare_methods()


In [ ]:
# Game parameters
K = 3  # Can choose 1, 2, or 3
N = 10  # Game ends when sum exceeds 10

# Create game and solver
game = AdditionGame(k=K, N=N)
solver = DynamicProgrammingSolver(k=K, N=N)

print(f"Solving Addition game with k={K}, N={N}")
print(f"State space: {N + 1} non-terminal states × 2 players = {2*(N+1)} states")
print(f"Action space: {K} actions per state")
print()

# Solve using Dynamic Programming
result = solver.solve(verbose=True)
